In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, GridSearchCV
# 网格搜索可以作为pipeline中的元素，从而进行搜索。
def Imbalance(estimator, grid_params, X, y):
    grid = GridSearchCV(estimator=estimator,grid_params=grid_params,
                       n_jobs=-1,cv=3)
    for train, test in KFold(5).split(X, y):
        pipeline=imbalanced_make_pipeline(
        SMOTE(sampling_strategy='minority'), grid)
        model = pipeline.fit(X[train], y[train])
        best = grid.best_estimator_
        prediction = best.predict(X[test])

The choice of small samples from the original dataset is based on the intuition that one class characteristics (non fraud) will differ from that of the other (fraud). To distinguish these characteristics we need to show the autoencoders only one class of data. This is because the autoencoder will try to learn only one class and automaticlly distinuish the other class. 

In [ ]:
from keras.layers import Input

from keras.layers import Input,Dense
import keras.regularizers as regularizers
from keras.models import Model
import sklearn.preprocessing as preprocessing

input_layer = Input(shape=(X.shape[1],))
encoded = Dense(100, activation='tanh', 
                activity_regularizer=regularizers.l1(10e-3))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)
output_layer = Dense(X.shape[1], activation='relu')(decoded)
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

x = data.drop(["Class"], axis=1)
y = data["Class"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]
autoencoder.fit(x_norm[0:2000], x_norm[0:2000], 
                batch_size = 256, epochs = 10, 
                shuffle = True, validation_split = 0.20);

'''Now, the model is trained. We are intereseted in obtaining 
latent representation of the input learned by the model.
This can be accessed by the weights of the trained model. 
We will create another network containing sequential layers, 
and we will only add the trained
weights till the third layer where latent representation exists. '''
from keras.models import Sequential
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

'''Generate the hidden representations of two classes : 
non-fraud and fraud by predicting the raw inputs using the above model.'''
norm_hid_rep = hidden_representation.predict(x_norm[:3000])
fraud_hid_rep = hidden_representation.predict(x_fraud)


rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

In [ ]:
# Separate majority and minority classes
df_majority = df_train[df_train.def_pay==0]
df_minority = df_train[df_train.def_pay==1]
# Upsample minority class
df_minority_upsampled = resample(df_minority,replace=True,   
                    n_samples=18677, random_state=587) # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
# Downsample majority class
df_majority_downsampled = resample(df_majority, replace=False,   
                 n_samples=5323, random_state=587) # reproducible results
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=589, ratio = 1.0)
X_SMOTE, y_SMOTE = sm.fit_sample(X_train, y_train)

In [ ]:
df = np.random.randn(1000,6)
df = pd.DataFrame(df,columns=list('ABCDEF'))

df.loc[:,:] = np.where(df>0,0,1)

df.columns

fig = plt.figure(figsize=(8,10))
ax = fig.add_subplot()
for i, feature in enumerate(df.columns):
    x = np.array([i]*df.shape[0])
    y = np.arange(df.shape[0])
    mask=df[feature]==0
    ax.scatter(x[~mask],y[~mask],c='k',s=1)
ax.xaxis.set_ticks(range(len(df.columns)))
ax.xaxis.set_ticklabels(df.columns);

from statsmodels.stats.outliers_influence import variance_inflation_factor
def VIF(data):
    data = data.to_numpy()
    res = [variance_inflation_factor(data, i) for i in range(data.shape[1])]
    s,v,d = np.linalg.svd(data)
    ratio = v.max()/v.min()
    return res, ratio